In [ ]:
!pip install langchain_experimental
!pip installlangchain_community
!pip install langchain_openai
!pip install ta

  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_community-0.3.16-py3-none-any.whl.metadata (2.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_experimental-0.3.4-py3-none-any.whl (209 kB)
Using cached langchain_community-0.3.16-py3-none-any.whl (2.5 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached pydantic_settings-2.7.1-py3-none-any.whl (29 kB)
Using cached marshmallow-3.26.1-py3-non

In [ ]:
!pip install langgraph

  Using cached langgraph-0.2.69-py3-none-any.whl.metadata (17 kB)
  Using cached langgraph_checkpoint-2.0.10-py3-none-any.whl.metadata (4.6 kB)
  Using cached langgraph_sdk-0.1.51-py3-none-any.whl.metadata (1.8 kB)
Using cached langgraph-0.2.69-py3-none-any.whl (148 kB)
Using cached langgraph_checkpoint-2.0.10-py3-none-any.whl (37 kB)
Using cached langgraph_sdk-0.1.51-py3-none-any.whl (44 kB)


In [ ]:
from langchain import hub
from langchain_experimental.utilities import PythonREPL
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, load_tools
from langchain.agents import Tool
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
from google.colab import userdata

DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

import os
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class Agent:

    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:
                print("\n ....bad tool name....")
                result = "bad tool name, retry"
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(
    model="deepseek-chat",
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com",
    temperature=0.0
)

tool = TavilySearchResults(max_results=4)

agent = Agent(model, [tool], system=prompt)

In [ ]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = agent.graph.invoke({"messages": messages})
print(result)
from IPython.display import Markdown
display(Markdown(result["messages"][-1].content))

In [ ]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = agent.graph.invoke({"messages": messages})
display(Markdown(result["messages"][-1].content))